# Taler de Arquitecturas de Datos
**🔬 Paso 1: Creando Nuestro Universo de Datos**

¡Bienvenidos al taller práctico! Antes de poder construir y comparar arquitecturas, necesitamos la materia prima: los datos.

En esta celda, ejecutaremos un script de PySpark que simula un ecosistema de datos completo para una empresa de E-commerce. Este no es un dataset estático; es un generador que creará un universo de datos controlado y realista para nuestro laboratorio.

**¿Qué Genera este Script?**
Este código creará varios "activos de datos" que simulan las diferentes fuentes que encontrarías en un entorno real:

* **Tablas Estructuradas:** usuarios, productos y pedidos, simulando los datos de una base de datos transaccional.
* **Logs Semi-Estructurados:** logs_web con la actividad de navegación de los usuarios.
* **Archivos No Estructurados:** Facturas en formato de texto guardadas en DBFS, simulando la ingesta de archivos como PDFs.
* **Documentos Complejos:** Un perfil_360 de cliente, simulando cómo se verían los datos en una base de datos NoSQL como Cosmos DB o MongoDB.

**Puntos Clave:**
* **Librería Faker:** Usamos esta librería para generar datos que parecen reales (nombres, emails, fechas, etc.).
* **Semilla de Reproducibilidad:** Hemos fijado una SEMILLA para que cada vez que se ejecute el script, genere exactamente los mismos datos. Elemental para que todos obtengamos los mismos resultados en los ejercicios.

**Acción:** Ejecuta esta celda para generar todos los DataFrames y archivos necesarios. ¡Este es el punto de partida para nuestro viaje a través de las arquitecturas de datos!


In [0]:
# Script para generar datos simulados para el Taller de Arquitecturas de Datos
# Este script utiliza PySpark y la librería Faker para crear datos realistas,
# incluyendo la simulación de archivos de texto (facturas) y logs web.
#
# Instrucciones en Databricks:
# 1. Asegúrate de que la librería 'Faker' esté instalada en tu cluster.
#    Puedes hacerlo a través de la UI del cluster en la pestaña "Libraries".
#    - PyPI -> package: Faker
# 2. Copia y pega este código en una celda de un notebook de Databricks.
# 3. Ejecuta la celda.


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType, ArrayType, MapType
from faker import Faker
import random
from datetime import datetime, timedelta
import json

# Inicializar Faker para generar datos falsos
fake = Faker('es_ES') # Usar localización en español para datos más realistas

# --- SEMILLA PARA REPRODUCIBILIDAD ---
# Establecemos una semilla para que los datos generados sean siempre los mismos en cada ejecución.
# Esto es crucial para que todos los estudiantes trabajen con el mismo dataset.
SEED = 42
Faker.seed(SEED)
random.seed(SEED)

# --- 1. Inicialización de Spark Session ---
# En un notebook de Databricks, la sesión de Spark ya está creada como 'spark'.
print("Spark Session iniciada.")

# --- 2. Funciones de Generación de Datos Relacionales ---

def generar_usuarios(n=100):
    """Genera una lista de diccionarios de usuarios."""
    data = []
    for i in range(n):
        data.append({
            'id_usuario': 1000 + i,
            'nombre': fake.name(),
            'email': fake.email(),
            'fecha_registro': fake.date_time_between(start_date='-2y', end_date='now'),
            'ciudad': fake.city()
        })
    return data

def generar_productos(n=50):
    """Genera una lista de diccionarios de productos."""
    categorias = ['Electrónica', 'Hogar', 'Ropa', 'Libros', 'Deportes']
    data = []
    for i in range(n):
        data.append({
            'id_producto': 2000 + i,
            'nombre_producto': fake.word().capitalize() + " " + fake.word(),
            'categoria': random.choice(categorias),
            'precio_unitario': round(random.uniform(5.0, 250.0), 2)
        })
    return data

def generar_pedidos(usuarios, productos, n=500):
    """Genera una lista de diccionarios de pedidos, vinculando usuarios y productos."""
    data = []
    for i in range(n):
        usuario = random.choice(usuarios)
        producto = random.choice(productos)
        cantidad = random.randint(1, 5)
        data.append({
            'id_pedido': 3000 + i,
            'id_usuario': usuario['id_usuario'],
            'id_producto': producto['id_producto'],
            'cantidad': cantidad,
            'monto': round(cantidad * producto['precio_unitario'], 2),
            'fecha_pedido': fake.date_time_between(start_date=usuario['fecha_registro'], end_date='now')
        })
    return data

def generar_logs_web(usuarios, n=2000):
    """Genera una lista de diccionarios de logs de visitas web más realistas."""
    paginas = ['/inicio', '/producto/detalle', '/carrito', '/checkout', '/perfil']
    metodos = ['GET', 'GET', 'GET', 'POST', 'GET']
    status = [200, 200, 200, 200, 404, 500]
    data = []
    for i in range(n):
        usuario = random.choice(usuarios)
        data.append({
            'id_log': 4000 + i,
            'id_usuario': usuario['id_usuario'],
            'pagina_visitada': random.choice(paginas),
            'metodo_http': random.choice(metodos),
            'codigo_estado': random.choice(status),
            'timestamp': fake.date_time_between(start_date=usuario['fecha_registro'], end_date='now')
        })
    return data

print("Funciones de generación de datos creadas.")

# --- 3. Creación de DataFrames de Spark ---

usuarios_data = generar_usuarios(100)
productos_data = generar_productos(50)
pedidos_data = generar_pedidos(usuarios_data, productos_data, 500)
logs_web_data = generar_logs_web(usuarios_data, 2000)

usuarios_df = spark.createDataFrame(usuarios_data)
productos_df = spark.createDataFrame(productos_data)
pedidos_df = spark.createDataFrame(pedidos_data)
logs_web_df = spark.createDataFrame(logs_web_data)

print("\n--- DataFrames Relacionales Creados ---")
usuarios_df.show(3)
productos_df.show(3)
pedidos_df.show(3)
logs_web_df.show(3, truncate=False)

# --- 4. Generación de Datos No Estructurados (Simulación de Archivos) ---

def generar_y_guardar_facturas_texto(pedidos, usuarios, productos, ruta_dbfs):
    """
    Simula la creación de archivos de facturas (como si fueran PDFs convertidos a texto).
    Guarda cada factura como un archivo .txt en la ruta de DBFS especificada.
    """
    print(f"\nGenerando archivos de facturas en la ruta: {ruta_dbfs}")
    
    # Crear un mapa de usuarios y productos para búsqueda fácil
    mapa_usuarios = {u['id_usuario']: u for u in usuarios}
    mapa_productos = {p['id_producto']: p for p in productos}
    
    # Asegurarse de que el directorio existe
    dbutils.fs.mkdirs(ruta_dbfs)
    
    # Tomar una muestra de 50 pedidos para generar facturas
    for pedido in random.sample(pedidos, 50):
        usuario = mapa_usuarios.get(pedido['id_usuario'])
        producto = mapa_productos.get(pedido['id_producto'])
        
        if not usuario or not producto:
            continue
            
        # Crear el contenido de la factura como un string
        contenido_factura = f"""
        ========================================
        FACTURA ELECTRÓNICA
        ========================================
        
        Número de Factura: INV-{pedido['id_pedido']}
        Fecha: {pedido['fecha_pedido'].strftime('%Y-%m-%d %H:%M:%S')}
        
        --- Cliente ---
        ID Cliente: {usuario['id_usuario']}
        Nombre: {usuario['nombre']}
        Email: {usuario['email']}
        Ciudad: {usuario['ciudad']}
        
        --- Detalles del Pedido ---
        ID Pedido: {pedido['id_pedido']}
        
        Descripción                 Cantidad      Precio Unit.      Total
        -----------------------------------------------------------------
        {producto['nombre_producto']:<28}{pedido['cantidad']:<14}${producto['precio_unitario']:<16.2f}${pedido['monto']:.2f}
        
        ========================================
        TOTAL A PAGAR: ${pedido['monto']:.2f}
        ========================================
        """
        
        # Guardar el string en un archivo en DBFS
        nombre_archivo = f"factura_{pedido['id_pedido']}.txt"
        dbutils.fs.put(f"{ruta_dbfs}/{nombre_archivo}", contenido_factura, overwrite=True)
        
    print(f"Se generaron 50 archivos de factura de ejemplo en {ruta_dbfs}")
    print("Los estudiantes pueden usar Auto Loader o spark.read.text() para ingerir estos datos.")

# Ejecutar la función para generar los archivos de factura
ruta_facturas = "/tmp/facturas_raw"
generar_y_guardar_facturas_texto(pedidos_data, usuarios_data, productos_data, ruta_facturas)


# --- 5. Función para Generar Documentos de Perfil 360 (Simulación NoSQL) ---

from pyspark.sql import functions as F

def crear_perfil_360(usuarios_df, pedidos_df, logs_web_df):
    """
    Combina los DataFrames para crear un perfil 360 de cada cliente (simulación de Cosmos DB/MongoDB).
    """
    pedidos_agrupados = pedidos_df.groupBy("id_usuario").agg(F.collect_list(F.struct("id_pedido", "id_producto", "monto", "fecha_pedido")).alias("pedidos"))
    logs_agrupados = logs_web_df.groupBy("id_usuario").agg(F.collect_list(F.struct("pagina_visitada", "timestamp")).alias("actividad_web"))
    
    perfil_360_df = usuarios_df.join(pedidos_agrupados, "id_usuario", "left").join(logs_agrupados, "id_usuario", "left")
    return perfil_360_df

print("\n--- Generando Documentos de Perfil 360 (Simulación NoSQL) ---")
perfil_360_df = crear_perfil_360(usuarios_df, pedidos_df, logs_web_df)

print("Mostrando una muestra de los perfiles 360:")
perfil_360_df.show(3, truncate=False)

# --- 6. Visualización de un Documento JSON ---

print("\n--- Ejemplo de un Documento JSON para el Perfil 360 ---")
primer_perfil_json = perfil_360_df.first().asDict(recursive=True)
print(json.dumps(primer_perfil_json, indent=4, default=str))

#
# Fin del Script
# Los DataFrames y los archivos de texto ya están listos para el taller.
#
